In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}\n\n")

Before ReLU: tensor([[-0.3548,  0.2507,  0.0040,  0.5837, -0.1234,  0.1426,  0.4402,  0.2701,
         -0.5740, -0.2093,  0.3992, -0.0955, -0.0695, -0.0335, -0.4017, -0.3873,
          0.1641, -0.4852,  0.1524, -0.1659],
        [-0.2450,  0.4680, -0.2010,  0.3538, -0.0518, -0.0025,  0.1470, -0.1235,
         -0.1582, -0.3256,  0.7525, -0.2134,  0.0967,  0.0479,  0.1329, -0.1427,
          0.1594, -0.2739,  0.1216, -0.2367],
        [-0.5847,  0.4204, -0.0084, -0.0169,  0.1253,  0.3058,  0.3127,  0.0326,
         -0.3995, -0.5000,  0.1048, -0.0600, -0.1992,  0.2909, -0.0786, -0.1209,
          0.1094, -0.4200, -0.2266, -0.0993]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2507, 0.0040, 0.5837, 0.0000, 0.1426, 0.4402, 0.2701, 0.0000,
         0.0000, 0.3992, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1641, 0.0000,
         0.1524, 0.0000],
        [0.0000, 0.4680, 0.0000, 0.3538, 0.0000, 0.0000, 0.1470, 0.0000, 0.0000,
         0.0000, 0.7525, 0.0000, 0.0967, 0.0479, 0.13

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0824, 0.0922, 0.1314, 0.1139, 0.0995, 0.0981, 0.0901, 0.1035, 0.1050,
         0.0839],
        [0.0750, 0.0983, 0.1187, 0.1200, 0.0901, 0.0923, 0.0975, 0.1056, 0.1093,
         0.0933],
        [0.0818, 0.0872, 0.1176, 0.1198, 0.1066, 0.0933, 0.0902, 0.1115, 0.0977,
         0.0941]], grad_fn=<SoftmaxBackward0>)


In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0172,  0.0031,  0.0343,  ...,  0.0104,  0.0228,  0.0198],
        [-0.0046, -0.0051, -0.0053,  ...,  0.0016,  0.0126,  0.0046]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0115, 0.0025], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0046, -0.0125, -0.0105,  ...,  0.0361,  0.0234,  0.0139],
        [ 0.0106, -0.0371, -0.0310,  ...,  0.0308,  0.0381,  0.0118]],
       device='cuda:0', grad_fn=<Slic